## Argo File Loader

In [5]:
import argopy
from argopy import DataFetcher as ArgoDataFetcher
argo_loader=ArgoDataFetcher(src='gdac',ftp="/swot/SUM05/dbalwada/202206-ArgoData",parallel=True)

In [12]:
box=[-35,-25,40,45,0,2000]

In [13]:
ds = argo_loader.region(box).to_xarray()

ValueError: 'PROFILE_PSAL_QC' is not present in all datasets.

In [8]:
ds=ds.argo.point2profile()

In [9]:
ds.to_netcdf("202206_lon:({},{})_lat:({},{}).nc".format(box[0],box[1],box[2],box[3]))